# Data preparation

## Read csv file

In [ ]:
import pandas as pd

df = pd.read_csv('assignment/loan_data_set.csv')
df.info()

## Fill empty cells

In [ ]:
fillna_mapping = {
    'LoanAmount': df['LoanAmount'].mean(skipna=True),
    'Gender': df['Gender'].mode(dropna=True).iloc[0],
    'Married': df['Married'].mode(dropna=True).iloc[0],
    'Dependents': df['Dependents'].mode(dropna=True).iloc[0],
    'Education': df['Education'].mode(dropna=True).iloc[0],
    'Self_Employed': df['Self_Employed'].mode(dropna=True).iloc[0],
    'ApplicantIncome': df['ApplicantIncome'].mode(dropna=True).iloc[0],
    'CoapplicantIncome': df['CoapplicantIncome'].mode(dropna=True).iloc[0],
    'Credit_History': df['Credit_History'].mode(dropna=True).iloc[0],
    'Loan_Amount_Term': df['Loan_Amount_Term'].mode(dropna=True).iloc[0],
}
df.fillna(fillna_mapping, inplace=True)
df.info()

## Encode labels

In [ ]:
label_mapping = {
    'Male': 0,
    'Female': 1,
    'No': 0,
    'Yes': 1,
    '0': 0,
    '1': 1,
    '2': 2,
    '3+': 3,
    'Not Graduate': 0,
    'Graduate': 1,
    'N': 0,
    'Y': 1,
}
df.replace(label_mapping, inplace=True)
# cast ApplicantIncome column to float
df['ApplicantIncome'] = df['ApplicantIncome'].astype(float)
df.info()

## Drop Loan_ID column

In [ ]:
df.drop('Loan_ID', axis='columns', inplace=True)
df.info()